In [1]:
from dataloader import get_dataloader
import torch
import numpy as np

In [2]:
# First find the unique phones in train.json, and then
# create a file named vocab.txt, each line in this 
# file is a unique phone, in total there should be 
# 40 lines

vocab = {}
phonemes = []
with open("vocab_39.txt") as f:
    for id, text in enumerate(f):
        vocab[text.strip()] = id
        phonemes.append(text)
phonemes = phonemes[1:]

In [3]:
from collections import namedtuple
if torch.cuda.is_available():
    device = "cuda:0"
    print("currently using cuda")
else:
    device = "cpu"
    print("currently using cpu only")

currently using cuda


In [4]:
dropout_rates = [0, 0.1, 0.3, 0.5]
Optimiser = ["SGD_Scheduler"]

In [5]:
import model_regularisation_dropout_between_layer
from datetime import datetime
from trainer_SGD_Scheduler import train as sgd_trainer
from trainer_Adam import train as adam_trainer
import torch
from decoder import decode

print("Start dropout tuning For 2 Layer LSTM, with Dropout between layer")

for opt in Optimiser:
    print("Currently using "+ opt +" optimiser")
    if opt=="Adam":
        args = {'seed': 123,
            'train_json': 'train_fbank_augmentation.json',
            'val_json': 'dev_fbank.json',
            'test_json': 'test_fbank.json',
            'batch_size': 4,
            'num_layers': 2,
            'fbank_dims': 23,
            'model_dims': 128,
            'concat': 1,
            'lr': 0.001,
            'vocab': vocab,
            'report_interval': 50,
            'num_epochs': 20,
            'device': device,
           }

        args = namedtuple('x', args)(**args)
    else:
        args = {'seed': 123,
            'train_json': 'train_fbank_augmentation.json',
            'val_json': 'dev_fbank.json',
            'test_json': 'test_fbank.json',
            'batch_size': 4,
            'num_layers': 2,
            'fbank_dims': 23,
            'model_dims': 128,
            'concat': 1,
            'lr': 0.5,
            'vocab': vocab,
            'report_interval': 50,
            'num_epochs': 20,
            'device': device,
           }

        args = namedtuple('x', args)(**args)
        
    
    for dropout_rate in dropout_rates:
        print("Currently using dropout rate of "+ str(dropout_rate))
        model_with_dropout = model_regularisation_dropout_between_layer.BiLSTM(2, args.fbank_dims * args.concat, args.model_dims, len(args.vocab), dropout_rate)
        num_params = sum(p.numel() for p in model_with_dropout.parameters())
        print('Total number of model parameters is {}'.format(num_params))
        start = datetime.now()
        model_with_dropout.to(args.device)
        if opt=="Adam":
            model_path = adam_trainer(model_with_dropout, args)
        else:
            model_path = sgd_trainer(model_with_dropout, args)
        end = datetime.now()
        duration = (end - start).total_seconds()
        print('Training finished in {} minutes.'.format(divmod(duration, 60)[0]))
        print('Model saved to {}'.format(model_path))
    
    print("Finish "+ opt +" optimiser")
print("End tuning For 2 Layer LSTM")
        
    
    

Start dropout tuning For 2 Layer LSTM, with Dropout between layer
Currently using SGD_Scheduler optimiser
Currently using dropout rate of 0
Total number of model parameters is 562216
EPOCH 1:
  batch 50 loss: 4.953719806671143
  batch 100 loss: 3.3993154096603395
  batch 150 loss: 3.300382390022278
  batch 200 loss: 3.1780371713638305
  batch 250 loss: 3.064969687461853
  batch 300 loss: 2.8370530080795286
  batch 350 loss: 2.6840357875823972
  batch 400 loss: 2.575184178352356
  batch 450 loss: 2.462020764350891
  batch 500 loss: 2.3452905082702635
  batch 550 loss: 2.2567735862731935
  batch 600 loss: 2.200996849536896
  batch 650 loss: 2.107234065532684
  batch 700 loss: 2.0393861436843874
  batch 750 loss: 1.9655284953117371
  batch 800 loss: 1.911541314125061
  batch 850 loss: 1.9214751029014587
  batch 900 loss: 1.8436394572257995
  batch 950 loss: 1.800590009689331
  batch 1000 loss: 1.7497910404205321
  batch 1050 loss: 1.6814700603485107
  batch 1100 loss: 1.6279713416099548
 

  batch 2250 loss: 0.6603223425149918
  batch 2300 loss: 0.6594460576772689
  batch 2350 loss: 0.6609499651193619
  batch 2400 loss: 0.6409342026710511
  batch 2450 loss: 0.6322945392131806
  batch 2500 loss: 0.686556544303894
  batch 2550 loss: 0.6657145524024963
  batch 2600 loss: 0.6866445565223693
  batch 2650 loss: 0.6462947624921799
  batch 2700 loss: 0.6766492688655853
  batch 2750 loss: 0.6341919189691544
LOSS train 0.63419 valid 0.83288, valid PER 25.81%
EPOCH 5:
  batch 50 loss: 0.6331386977434158
  batch 100 loss: 0.6089944469928742
  batch 150 loss: 0.6204289311170578
  batch 200 loss: 0.6151063221693039
  batch 250 loss: 0.579046545624733
  batch 300 loss: 0.6445787811279297
  batch 350 loss: 0.6220481461286544
  batch 400 loss: 0.5797956824302674
  batch 450 loss: 0.6365955978631973
  batch 500 loss: 0.6200397503376007
  batch 550 loss: 0.6106108516454697
  batch 600 loss: 0.5838797509670257
  batch 650 loss: 0.6128009176254272
  batch 700 loss: 0.6067919433116913
  batch

  batch 1750 loss: 0.3581231477856636
  batch 1800 loss: 0.3444445192813873
  batch 1850 loss: 0.3627458393573761
  batch 1900 loss: 0.35728987425565717
  batch 1950 loss: 0.3488490375876427
  batch 2000 loss: 0.33862262666225434
  batch 2050 loss: 0.367991014122963
  batch 2100 loss: 0.3611968460679054
  batch 2150 loss: 0.3819046404957771
  batch 2200 loss: 0.3519177129864693
  batch 2250 loss: 0.3503365993499756
  batch 2300 loss: 0.35073968917131426
  batch 2350 loss: 0.36448561400175095
  batch 2400 loss: 0.3584690853953362
  batch 2450 loss: 0.32283335238695143
  batch 2500 loss: 0.3499716010689735
  batch 2550 loss: 0.36583689600229263
  batch 2600 loss: 0.34491838842630385
  batch 2650 loss: 0.35270298421382906
  batch 2700 loss: 0.3336675837635994
  batch 2750 loss: 0.363292818069458
LOSS train 0.36329 valid 0.77704, valid PER 22.59%
EPOCH 9:
  batch 50 loss: 0.3169567310810089
  batch 100 loss: 0.30438849300146104
  batch 150 loss: 0.30472632229328156
  batch 200 loss: 0.3110

  batch 950 loss: 0.18678005650639534
  batch 1000 loss: 0.18697609066963194
  batch 1050 loss: 0.18464520432054995
  batch 1100 loss: 0.17821577459573745
  batch 1150 loss: 0.18456487104296684
  batch 1200 loss: 0.18207444801926612
  batch 1250 loss: 0.18611753433942796
  batch 1300 loss: 0.17698888897895812
  batch 1350 loss: 0.1901748187839985
  batch 1400 loss: 0.18531769126653672
  batch 1450 loss: 0.18777516454458237
  batch 1500 loss: 0.19808893144130707
  batch 1550 loss: 0.1907767315208912
  batch 1600 loss: 0.18621637269854546
  batch 1650 loss: 0.20473864793777466
  batch 1700 loss: 0.19696079179644585
  batch 1750 loss: 0.17612159952521325
  batch 1800 loss: 0.17171434685587883
  batch 1850 loss: 0.18141204185783863
  batch 1900 loss: 0.19442686542868615
  batch 1950 loss: 0.1837778042256832
  batch 2000 loss: 0.18425996944308282
  batch 2050 loss: 0.18283584341406822
  batch 2100 loss: 0.18681820705533028
  batch 2150 loss: 0.17854492127895355
  batch 2200 loss: 0.17579241

  batch 50 loss: 0.1754765547811985
  batch 100 loss: 0.1617314687371254
  batch 150 loss: 0.17069957703351973
  batch 200 loss: 0.1602345472574234
  batch 250 loss: 0.16497291818261148
  batch 300 loss: 0.165514215528965
  batch 350 loss: 0.16981702148914338
  batch 400 loss: 0.1668749901652336
  batch 450 loss: 0.15690707445144653
  batch 500 loss: 0.17490482419729234
  batch 550 loss: 0.15982845827937125
  batch 600 loss: 0.161205098554492
  batch 650 loss: 0.17022019132971763
  batch 700 loss: 0.16750350549817086
  batch 750 loss: 0.15492497146129608
  batch 800 loss: 0.16567156627774238
  batch 850 loss: 0.15849783539772033
  batch 900 loss: 0.1612556102871895
  batch 950 loss: 0.1466603533178568
  batch 1000 loss: 0.17151930853724479
  batch 1050 loss: 0.17835172563791274
  batch 1100 loss: 0.16304952129721642
  batch 1150 loss: 0.16401251137256623
  batch 1200 loss: 0.15943723246455194
  batch 1250 loss: 0.1645649553835392
  batch 1300 loss: 0.17839066058397293
  batch 1350 loss

...

  batch 900 loss: 0.5330305236577988
  batch 950 loss: 0.5303064233064652
  batch 1000 loss: 0.5467915225028992
  batch 1050 loss: 0.5234055888652801
  batch 1100 loss: 0.5265372186899185
  batch 1150 loss: 0.5394180148839951
  batch 1200 loss: 0.5490663731098175
  batch 1250 loss: 0.5588401943445206
  batch 1300 loss: 0.5709744572639466
  batch 1350 loss: 0.4965359193086624
  batch 1400 loss: 0.5183021306991578
  batch 1450 loss: 0.5279060959815979
  batch 1500 loss: 0.5013817924261094
  batch 1550 loss: 0.5444025576114655
  batch 1600 loss: 0.5368912118673325
  batch 1650 loss: 0.5152462863922119
  batch 1700 loss: 0.5503312271833419
  batch 1750 loss: 0.4896565079689026
  batch 1800 loss: 0.5111396026611328
  batch 1850 loss: 0.5419363248348236
  batch 1900 loss: 0.5064775037765503
  batch 1950 loss: 0.5310086888074875
  batch 2000 loss: 0.5015270274877548
  batch 2050 loss: 0.5441862630844116
  batch 2100 loss: 0.534551482796669
  batch 2150 loss: 0.5290987724065781
  batch 2200 lo

  batch 250 loss: 0.27728149145841596
  batch 300 loss: 0.2895902115106583
  batch 350 loss: 0.2856204405426979
  batch 400 loss: 0.28121084690093995
  batch 450 loss: 0.27859010845422744
  batch 500 loss: 0.2859617409110069
  batch 550 loss: 0.28804107785224914
  batch 600 loss: 0.28548723191022873
  batch 650 loss: 0.2630582246184349
  batch 700 loss: 0.26530607134103773
  batch 750 loss: 0.27682259410619736
  batch 800 loss: 0.277895981669426
  batch 850 loss: 0.2859682211279869
  batch 900 loss: 0.2621074056625366
  batch 950 loss: 0.27790908694267275
  batch 1000 loss: 0.2765858566761017
  batch 1050 loss: 0.24775049477815628
  batch 1100 loss: 0.26967536717653273
  batch 1150 loss: 0.26703612983226777
  batch 1200 loss: 0.27961874663829805
  batch 1250 loss: 0.283763110935688
  batch 1300 loss: 0.2694987803697586
  batch 1350 loss: 0.2710930714011192
  batch 1400 loss: 0.25163019984960555
  batch 1450 loss: 0.27653350442647934
  batch 1500 loss: 0.2542334453761578
  batch 1550 lo

  batch 2250 loss: 0.20834460407495498
  batch 2300 loss: 0.20901915460824966
  batch 2350 loss: 0.20926114723086356
  batch 2400 loss: 0.224500602632761
  batch 2450 loss: 0.20586979925632476
  batch 2500 loss: 0.2143600106239319
  batch 2550 loss: 0.22060076966881753
  batch 2600 loss: 0.20150316804647445
  batch 2650 loss: 0.20638425692915915
  batch 2700 loss: 0.19934888258576394
  batch 2750 loss: 0.22576652273535727
Epoch 00014: reducing learning rate of group 0 to 7.8125e-03.
LOSS train 0.22577 valid 0.83286, valid PER 22.10%
EPOCH 15:
  batch 50 loss: 0.20025224804878236
  batch 100 loss: 0.20949839562177658
  batch 150 loss: 0.2195408633351326
  batch 200 loss: 0.20798778608441354
  batch 250 loss: 0.19043301522731781
  batch 300 loss: 0.21326522141695023
  batch 350 loss: 0.20282391235232353
  batch 400 loss: 0.2022451750934124
  batch 450 loss: 0.2127455660700798
  batch 500 loss: 0.21037504807114601
  batch 550 loss: 0.20124362409114838
  batch 600 loss: 0.21857730016112328

  batch 1350 loss: 0.1993356542289257
  batch 1400 loss: 0.20398598209023475
  batch 1450 loss: 0.1956988087296486
  batch 1500 loss: 0.19626488134264947
  batch 1550 loss: 0.1974722048640251
  batch 1600 loss: 0.20159361451864244
  batch 1650 loss: 0.1965295284986496
  batch 1700 loss: 0.20830517157912254
  batch 1750 loss: 0.19299522206187247
  batch 1800 loss: 0.19528258442878724
  batch 1850 loss: 0.20994090750813485
  batch 1900 loss: 0.20032187655568123
  batch 1950 loss: 0.20334867745637894
  batch 2000 loss: 0.22872849866747857
  batch 2050 loss: 0.19549178957939148
  batch 2100 loss: 0.21772972300648688
  batch 2150 loss: 0.20159623995423318
  batch 2200 loss: 0.1999000833928585
  batch 2250 loss: 0.2070457910001278
  batch 2300 loss: 0.19337164163589476
  batch 2350 loss: 0.20964196249842643
  batch 2400 loss: 0.21048405185341834
  batch 2450 loss: 0.2001415039598942
  batch 2500 loss: 0.18920308127999305
  batch 2550 loss: 0.20626488849520683
  batch 2600 loss: 0.19085694864

KeyboardInterrupt: 

In [5]:
dropout_rates = [0.5]
Optimiser = ["SGD_Scheduler"]

In [6]:
import model_regularisation_dropout_between_layer
from datetime import datetime
from trainer_SGD_Scheduler import train as sgd_trainer
from trainer_Adam import train as adam_trainer
import torch
from decoder import decode

print("Start dropout tuning For 2 Layer LSTM, with Dropout between layer")

for opt in Optimiser:
    print("Currently using "+ opt +" optimiser")
    if opt=="Adam":
        args = {'seed': 123,
            'train_json': 'train_fbank_augmentation.json',
            'val_json': 'dev_fbank.json',
            'test_json': 'test_fbank.json',
            'batch_size': 4,
            'num_layers': 2,
            'fbank_dims': 23,
            'model_dims': 128,
            'concat': 1,
            'lr': 0.001,
            'vocab': vocab,
            'report_interval': 50,
            'num_epochs': 20,
            'device': device,
           }

        args = namedtuple('x', args)(**args)
    else:
        args = {'seed': 123,
            'train_json': 'train_fbank_augmentation.json',
            'val_json': 'dev_fbank.json',
            'test_json': 'test_fbank.json',
            'batch_size': 4,
            'num_layers': 2,
            'fbank_dims': 23,
            'model_dims': 128,
            'concat': 1,
            'lr': 0.5,
            'vocab': vocab,
            'report_interval': 50,
            'num_epochs': 20,
            'device': device,
           }

        args = namedtuple('x', args)(**args)
        
    
    for dropout_rate in dropout_rates:
        print("Currently using dropout rate of "+ str(dropout_rate))
        model_with_dropout = model_regularisation_dropout_between_layer.BiLSTM(2, args.fbank_dims * args.concat, args.model_dims, len(args.vocab), dropout_rate)
        num_params = sum(p.numel() for p in model_with_dropout.parameters())
        print('Total number of model parameters is {}'.format(num_params))
        start = datetime.now()
        model_with_dropout.to(args.device)
        if opt=="Adam":
            model_path = adam_trainer(model_with_dropout, args)
        else:
            model_path = sgd_trainer(model_with_dropout, args)
        end = datetime.now()
        duration = (end - start).total_seconds()
        print('Training finished in {} minutes.'.format(divmod(duration, 60)[0]))
        print('Model saved to {}'.format(model_path))
    
    print("Finish "+ opt +" optimiser")
print("End tuning For 2 Layer LSTM")
        
    
    

Start dropout tuning For 2 Layer LSTM, with Dropout between layer
Currently using SGD_Scheduler optimiser
Currently using dropout rate of 0.5
Total number of model parameters is 562216
EPOCH 1:
  batch 50 loss: 4.963497977256775
  batch 100 loss: 3.384019055366516
  batch 150 loss: 3.299621024131775
  batch 200 loss: 3.1785534524917605
  batch 250 loss: 3.0771847105026247
  batch 300 loss: 2.8530867671966553
  batch 350 loss: 2.684001531600952
  batch 400 loss: 2.5836263513565063
  batch 450 loss: 2.481904878616333
  batch 500 loss: 2.361795382499695
  batch 550 loss: 2.2778086948394773
  batch 600 loss: 2.2288330817222595
  batch 650 loss: 2.1413475918769835
  batch 700 loss: 2.075978944301605
  batch 750 loss: 1.999926404953003
  batch 800 loss: 1.9528973960876466
  batch 850 loss: 1.9586704325675965
  batch 900 loss: 1.8926297879219056
  batch 950 loss: 1.8473450422286988
  batch 1000 loss: 1.8088407802581787
  batch 1050 loss: 1.7400964522361755
  batch 1100 loss: 1.670434269905090

  batch 2250 loss: 0.7382354551553726
  batch 2300 loss: 0.7816479676961898
  batch 2350 loss: 0.7619472283124924
  batch 2400 loss: 0.7660547268390655
  batch 2450 loss: 0.7867212772369385
  batch 2500 loss: 0.791603547334671
  batch 2550 loss: 0.7764487832784652
  batch 2600 loss: 0.8015234822034836
  batch 2650 loss: 0.8040702295303345
  batch 2700 loss: 0.7320561671257019
  batch 2750 loss: 0.7660239523649216
LOSS train 0.76602 valid 0.82979, valid PER 25.86%
EPOCH 5:
  batch 50 loss: 0.7538781034946441
  batch 100 loss: 0.7248894530534744
  batch 150 loss: 0.76694344997406
  batch 200 loss: 0.7351259028911591
  batch 250 loss: 0.738396846652031
  batch 300 loss: 0.7325045424699783
  batch 350 loss: 0.7741005194187164
  batch 400 loss: 0.7429319965839386
  batch 450 loss: 0.7318109476566315
  batch 500 loss: 0.7649670577049256
  batch 550 loss: 0.714766366481781
  batch 600 loss: 0.7244398188591004
  batch 650 loss: 0.7246204060316086
  batch 700 loss: 0.7715640312433243
  batch 75

  batch 1900 loss: 0.6263420355319976
  batch 1950 loss: 0.5834772741794586
  batch 2000 loss: 0.6091418641805649
  batch 2050 loss: 0.579633520245552
  batch 2100 loss: 0.5923056781291962
  batch 2150 loss: 0.6093237096071243
  batch 2200 loss: 0.5883041900396347
  batch 2250 loss: 0.6148463207483291
  batch 2300 loss: 0.5728337800502777
  batch 2350 loss: 0.5879289954900742
  batch 2400 loss: 0.601286627650261
  batch 2450 loss: 0.5736616915464401
  batch 2500 loss: 0.6104408544301987
  batch 2550 loss: 0.5910945773124695
  batch 2600 loss: 0.5963704586029053
  batch 2650 loss: 0.592735840678215
  batch 2700 loss: 0.6088889950513839
  batch 2750 loss: 0.6000488913059234
Epoch 00008: reducing learning rate of group 0 to 2.5000e-01.
LOSS train 0.60005 valid 0.75213, valid PER 22.73%
EPOCH 9:
  batch 50 loss: 0.5399199670553207
  batch 100 loss: 0.5524868553876877
  batch 150 loss: 0.5169765043258667
  batch 200 loss: 0.5265405505895615
  batch 250 loss: 0.5298130804300308
  batch 300 l

  batch 1250 loss: 0.4068649035692215
  batch 1300 loss: 0.43343608528375627
  batch 1350 loss: 0.43632018446922305
  batch 1400 loss: 0.42499372124671936
  batch 1450 loss: 0.43269784152507784
  batch 1500 loss: 0.4307167273759842
  batch 1550 loss: 0.418338857293129
  batch 1600 loss: 0.41596662759780884
  batch 1650 loss: 0.40908138394355775
  batch 1700 loss: 0.4234920519590378
  batch 1750 loss: 0.41847546517848966
  batch 1800 loss: 0.4264876502752304
  batch 1850 loss: 0.40391402870416643
  batch 1900 loss: 0.3945696699619293
  batch 1950 loss: 0.42258731603622435
  batch 2000 loss: 0.41524751782417296
  batch 2050 loss: 0.4119496649503708
  batch 2100 loss: 0.40770502388477325
  batch 2150 loss: 0.38744706034660337
  batch 2200 loss: 0.4301250886917114
  batch 2250 loss: 0.43837101966142655
  batch 2300 loss: 0.41147362619638445
  batch 2350 loss: 0.41714817106723784
  batch 2400 loss: 0.39847392082214356
  batch 2450 loss: 0.4038366425037384
  batch 2500 loss: 0.40244147866964

  batch 400 loss: 0.36928529292345047
  batch 450 loss: 0.3622478672862053
  batch 500 loss: 0.3770082339644432
  batch 550 loss: 0.3708686423301697
  batch 600 loss: 0.3504039615392685
  batch 650 loss: 0.37198147624731065
  batch 700 loss: 0.37444022089242934
  batch 750 loss: 0.3711646890640259
  batch 800 loss: 0.3421926686167717
  batch 850 loss: 0.37175315409898757
  batch 900 loss: 0.3494886839389801
  batch 950 loss: 0.35920357644557954
  batch 1000 loss: 0.3823190712928772
  batch 1050 loss: 0.3822011977434158
  batch 1100 loss: 0.3579691731929779
  batch 1150 loss: 0.3594176024198532
  batch 1200 loss: 0.3632941606640816
  batch 1250 loss: 0.34833808958530427
  batch 1300 loss: 0.38745873838663103
  batch 1350 loss: 0.37180380582809447
  batch 1400 loss: 0.35058013290166856
  batch 1450 loss: 0.3843952560424805
  batch 1500 loss: 0.3616779759526253
  batch 1550 loss: 0.3609975931048393
  batch 1600 loss: 0.3543637138605118
  batch 1650 loss: 0.3709777924418449
  batch 1700 lo

  batch 2450 loss: 0.3561384007334709
  batch 2500 loss: 0.36608962804079054
  batch 2550 loss: 0.36982079952955244
  batch 2600 loss: 0.3769643864035606
  batch 2650 loss: 0.37314257591962813
  batch 2700 loss: 0.3430856996774673
  batch 2750 loss: 0.36600650936365126
Epoch 00019: reducing learning rate of group 0 to 4.8828e-04.
LOSS train 0.36601 valid 0.70800, valid PER 20.59%
EPOCH 20:
  batch 50 loss: 0.34897023886442186
  batch 100 loss: 0.37523879170417784
  batch 150 loss: 0.33800047904253006
  batch 200 loss: 0.3840277177095413
  batch 250 loss: 0.3739973717927933
  batch 300 loss: 0.35727009177207947
  batch 350 loss: 0.36684033334255217
  batch 400 loss: 0.33930872023105624
  batch 450 loss: 0.3568574702739716
  batch 500 loss: 0.3443778231739998
  batch 550 loss: 0.3566037997603416
  batch 600 loss: 0.36193386137485506
  batch 650 loss: 0.34851242214441297
  batch 700 loss: 0.35787448912858966
  batch 750 loss: 0.3591548675298691
  batch 800 loss: 0.36566447019577025
  batc

In [1]:
from dataloader import get_dataloader
import torch
import numpy as np
# First find the unique phones in train.json, and then
# create a file named vocab.txt, each line in this 
# file is a unique phone, in total there should be 
# 40 lines

vocab = {}
phonemes = []
with open("vocab_39.txt") as f:
    for id, text in enumerate(f):
        vocab[text.strip()] = id
        phonemes.append(text)
phonemes = phonemes[1:]
from collections import namedtuple
if torch.cuda.is_available():
    device = "cuda:0"
    print("currently using cuda")
else:
    device = "cpu"
    print("currently using cpu only")

args = {'seed': 123,
        'train_json': 'train_fbank.json',
        'val_json': 'dev_fbank.json',
        'test_json': 'test_fbank.json',
        'batch_size': 4,
        'num_layers': 1,
        'fbank_dims': 23,
        'model_dims': 128,
        'concat': 1,
        'lr': 0.5,
        'vocab': vocab,
        'report_interval': 50,
        'num_epochs': 20,
        'device': device,
       }

args = namedtuple('x', args)(**args)

currently using cpu only


In [2]:
### You can uncomment the following line and change model path to the model you want to decode
model_path="checkpoints/20231211_145126/model_11"

In [3]:
import model_regularisation_dropout_between_layer
args = {'seed': 123,
            'train_json': 'train_fbank.json',
            'val_json': 'dev_fbank.json',
            'test_json': 'test_fbank.json',
            'batch_size': 4,
            'num_layers': 2,
            'fbank_dims': 23,
            'model_dims': 128,
            'concat': 1,
            'lr': 0.5,
            'vocab': vocab,
            'report_interval': 50,
            'num_epochs': 20,
            'device': device,
           }

args = namedtuple('x', args)(**args)
model = model_regularisation_dropout_between_layer.BiLSTM(2, args.fbank_dims * args.concat, args.model_dims, len(args.vocab),0)

In [4]:
import torch
print('Loading model from {}'.format(model_path))
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

Loading model from checkpoints/20231211_145126/model_11


BiLSTM(
  (lstm): LSTM(23, 128, num_layers=2, bidirectional=True)
  (proj): Linear(in_features=256, out_features=40, bias=True)
)

In [5]:
from decoder import decode
results = decode(model, args, args.test_json)
print("SUB: {:.2f}%, DEL: {:.2f}%, INS: {:.2f}%, COR: {:.2f}%, PER: {:.2f}%".format(*results))

SUB: 13.71%, DEL: 6.27%, INS: 2.78%, COR: 80.02%, PER: 22.76%


# 一天天就知道凶人的第四组实验

# Droupout rate 0.5
# SGD shceduler learning rate 0.7

In [1]:
from dataloader import get_dataloader
import torch
import numpy as np

In [2]:
# First find the unique phones in train.json, and then
# create a file named vocab.txt, each line in this 
# file is a unique phone, in total there should be 
# 40 lines

vocab = {}
phonemes = []
with open("vocab_39.txt") as f:
    for id, text in enumerate(f):
        vocab[text.strip()] = id
        phonemes.append(text)
phonemes = phonemes[1:]

In [3]:
from collections import namedtuple
if torch.cuda.is_available():
    device = "cuda:0"
    print("currently using cuda")
else:
    device = "cpu"
    print("currently using cpu only")

currently using cuda


In [4]:
dropout_rates = [0.5]
Optimiser = ["SGD_Scheduler"]

In [5]:
import model_regularisation_dropout_between_layer
from datetime import datetime
from trainer_SGD_Scheduler import train as sgd_trainer
from trainer_Adam import train as adam_trainer
import torch
from decoder import decode

print("Start dropout tuning For 2 Layer LSTM, with Dropout between layer")

for opt in Optimiser:
    print("Currently using "+ opt +" optimiser")
    if opt=="Adam":
        args = {'seed': 123,
            'train_json': 'train_fbank_augmentation.json',
            'val_json': 'dev_fbank.json',
            'test_json': 'test_fbank.json',
            'batch_size': 4,
            'num_layers': 2,
            'fbank_dims': 23,
            'model_dims': 128,
            'concat': 1,
            'lr': 0.001,
            'vocab': vocab,
            'report_interval': 50,
            'num_epochs': 20,
            'device': device,
           }

        args = namedtuple('x', args)(**args)
    else:
        args = {'seed': 123,
            'train_json': 'train_fbank_augmentation.json',
            'val_json': 'dev_fbank.json',
            'test_json': 'test_fbank.json',
            'batch_size': 4,
            'num_layers': 2,
            'fbank_dims': 23,
            'model_dims': 128,
            'concat': 1,
            'lr': 1.0,
            'vocab': vocab,
            'report_interval': 50,
            'num_epochs': 20,
            'device': device,
           }

        args = namedtuple('x', args)(**args)
        
    
    for dropout_rate in dropout_rates:
        print("Currently using dropout rate of "+ str(dropout_rate))
        model_with_dropout = model_regularisation_dropout_between_layer.BiLSTM(2, args.fbank_dims * args.concat, args.model_dims, len(args.vocab), dropout_rate)
        num_params = sum(p.numel() for p in model_with_dropout.parameters())
        print('Total number of model parameters is {}'.format(num_params))
        start = datetime.now()
        model_with_dropout.to(args.device)
        if opt=="Adam":
            model_path = adam_trainer(model_with_dropout, args)
        else:
            model_path = sgd_trainer(model_with_dropout, args)
        end = datetime.now()
        duration = (end - start).total_seconds()
        print('Training finished in {} minutes.'.format(divmod(duration, 60)[0]))
        print('Model saved to {}'.format(model_path))
    
    print("Finish "+ opt +" optimiser")
print("End tuning For 2 Layer LSTM")
        
    
    

Start dropout tuning For 2 Layer LSTM, with Dropout between layer
Currently using SGD_Scheduler optimiser
Currently using dropout rate of 0.5
Total number of model parameters is 562216
EPOCH 1:
  batch 50 loss: 4.609509544372559
  batch 100 loss: 3.2777310514450075
  batch 150 loss: 3.1288997316360474
  batch 200 loss: 2.799676470756531
  batch 250 loss: 2.629464559555054
  batch 300 loss: 2.448464527130127
  batch 350 loss: 2.368602070808411
  batch 400 loss: 2.291487956047058
  batch 450 loss: 2.162491626739502
  batch 500 loss: 2.063810396194458
  batch 550 loss: 1.9613248252868651
  batch 600 loss: 1.9027792024612427
  batch 650 loss: 1.834680082798004
  batch 700 loss: 1.7801838207244873
  batch 750 loss: 1.7071974110603332
  batch 800 loss: 1.6724721002578735
  batch 850 loss: 1.6971648359298706
  batch 900 loss: 1.6312711763381957
  batch 950 loss: 1.613643250465393
  batch 1000 loss: 1.583342628479004
  batch 1050 loss: 1.5154249572753906
  batch 1100 loss: 1.4750522994995117
 

  batch 2250 loss: 0.7091731494665146
  batch 2300 loss: 0.7480642765760421
  batch 2350 loss: 0.7344293802976608
  batch 2400 loss: 0.731310847401619
  batch 2450 loss: 0.7389164853096009
  batch 2500 loss: 0.7634649932384491
  batch 2550 loss: 0.7452573519945145
  batch 2600 loss: 0.7766497772932053
  batch 2650 loss: 0.7569308924674988
  batch 2700 loss: 0.7217086529731751
  batch 2750 loss: 0.7362372517585755
LOSS train 0.73624 valid 0.80403, valid PER 24.56%
EPOCH 5:
  batch 50 loss: 0.7183885109424591
  batch 100 loss: 0.7048973381519318
  batch 150 loss: 0.7154817104339599
  batch 200 loss: 0.7148961740732193
  batch 250 loss: 0.692699835896492
  batch 300 loss: 0.7222865521907806
  batch 350 loss: 0.739528289437294
  batch 400 loss: 0.6970810341835022
  batch 450 loss: 0.6971015375852585
  batch 500 loss: 0.7457836675643921
  batch 550 loss: 0.6872825926542282
  batch 600 loss: 0.703405739068985
  batch 650 loss: 0.6991213995218277
  batch 700 loss: 0.7376431119441986
  batch 7

  batch 1900 loss: 0.6102788025140762
  batch 1950 loss: 0.5813162809610367
  batch 2000 loss: 0.5789337688684464
  batch 2050 loss: 0.5748624390363694
  batch 2100 loss: 0.5952368766069412
  batch 2150 loss: 0.5845299732685089
  batch 2200 loss: 0.581552860736847
  batch 2250 loss: 0.5976512950658798
  batch 2300 loss: 0.583308116197586
  batch 2350 loss: 0.5868609386682511
  batch 2400 loss: 0.5774581253528595
  batch 2450 loss: 0.5500998955965042
  batch 2500 loss: 0.5901757258176804
  batch 2550 loss: 0.5864786577224731
  batch 2600 loss: 0.573754026889801
  batch 2650 loss: 0.5658316117525101
  batch 2700 loss: 0.6051390582323074
  batch 2750 loss: 0.5844399404525756
Epoch 00008: reducing learning rate of group 0 to 5.0000e-01.
LOSS train 0.58444 valid 0.77965, valid PER 23.18%
EPOCH 9:
  batch 50 loss: 0.531273792386055
  batch 100 loss: 0.5372254604101181
  batch 150 loss: 0.505369490981102
  batch 200 loss: 0.5084722268581391
  batch 250 loss: 0.48896885335445406
  batch 300 lo

  batch 1250 loss: 0.3650440439581871
  batch 1300 loss: 0.35872744739055634
  batch 1350 loss: 0.3750202488899231
  batch 1400 loss: 0.36659381568431854
  batch 1450 loss: 0.37849836975336076
  batch 1500 loss: 0.3701036471128464
  batch 1550 loss: 0.36179919451475145
  batch 1600 loss: 0.3550183042883873
  batch 1650 loss: 0.35201298773288725
  batch 1700 loss: 0.34569903552532194
  batch 1750 loss: 0.353071391582489
  batch 1800 loss: 0.359297431409359
  batch 1850 loss: 0.3382202282547951
  batch 1900 loss: 0.3508001750707626
  batch 1950 loss: 0.37884748697280884
  batch 2000 loss: 0.3603817707300186
  batch 2050 loss: 0.3463579028844833
  batch 2100 loss: 0.3604710686206818
  batch 2150 loss: 0.3342709872126579
  batch 2200 loss: 0.3646461334824562
  batch 2250 loss: 0.38544065952301027
  batch 2300 loss: 0.3493233835697174
  batch 2350 loss: 0.37590689957141876
  batch 2400 loss: 0.34440348446369173
  batch 2450 loss: 0.37044745445251465
  batch 2500 loss: 0.34114893823862075
  

  batch 350 loss: 0.2966096645593643
  batch 400 loss: 0.312538258433342
  batch 450 loss: 0.3097683009505272
  batch 500 loss: 0.2989265415072441
  batch 550 loss: 0.30231568694114686
  batch 600 loss: 0.293539477288723
  batch 650 loss: 0.3082249069213867
  batch 700 loss: 0.326841002702713
  batch 750 loss: 0.3145700228214264
  batch 800 loss: 0.2812103289365768
  batch 850 loss: 0.30074149429798125
  batch 900 loss: 0.3004875096678734
  batch 950 loss: 0.2937307956814766
  batch 1000 loss: 0.31693636953830717
  batch 1050 loss: 0.29798136264085767
  batch 1100 loss: 0.3015220034122467
  batch 1150 loss: 0.30834407806396485
  batch 1200 loss: 0.3042308434844017
  batch 1250 loss: 0.27773296505212786
  batch 1300 loss: 0.3282755324244499
  batch 1350 loss: 0.30770562052726746
  batch 1400 loss: 0.2931290239095688
  batch 1450 loss: 0.3077496710419655
  batch 1500 loss: 0.31754421412944794
  batch 1550 loss: 0.2962141454219818
  batch 1600 loss: 0.3088872143626213
  batch 1650 loss: 0

  batch 2400 loss: 0.29562187135219575
  batch 2450 loss: 0.3056300845742226
  batch 2500 loss: 0.3014137315750122
  batch 2550 loss: 0.28971683233976364
  batch 2600 loss: 0.30888008236885073
  batch 2650 loss: 0.2983162260055542
  batch 2700 loss: 0.28459420770406724
  batch 2750 loss: 0.3071652027964592
Epoch 00019: reducing learning rate of group 0 to 9.7656e-04.
LOSS train 0.30717 valid 0.73230, valid PER 20.02%
EPOCH 20:
  batch 50 loss: 0.28951789617538454
  batch 100 loss: 0.3122708675265312
  batch 150 loss: 0.2747110638022423
  batch 200 loss: 0.3168734005093575
  batch 250 loss: 0.31890604496002195
  batch 300 loss: 0.29542651802301406
  batch 350 loss: 0.3150186184048653
  batch 400 loss: 0.2852684712409973
  batch 450 loss: 0.2809467366337776
  batch 500 loss: 0.2909441965818405
  batch 550 loss: 0.2859696829319
  batch 600 loss: 0.30257520735263826
  batch 650 loss: 0.29372184187173844
  batch 700 loss: 0.299716919362545
  batch 750 loss: 0.28172645062208174
  batch 800 l